In [ ]:
from google.colab import drive
drive.mount('/content/gdrive', force_remount=True)

Mounted at /content/gdrive


In [ ]:
#from google.colab import drive
drive.mount('/content/gdrive', force_remount=True)

Mounted at /content/gdrive


In [ ]:
# Fix randomness and hide warnings
seed = 42

import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'
os.environ['PYTHONHASHSEED'] = str(seed)
os.environ['MPLCONFIGDIR'] = os.getcwd()+'/configs/'

import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
warnings.simplefilter(action='ignore', category=Warning)

import numpy as np
np.random.seed(seed)

import logging

import random
random.seed(seed)

# Import tensorflow
import tensorflow as tf
from tensorflow import keras as tfk
from tensorflow.keras import layers as tfkl
tf.autograph.set_verbosity(0)
tf.get_logger().setLevel(logging.ERROR)
tf.compat.v1.logging.set_verbosity(tf.compat.v1.logging.ERROR)
tf.random.set_seed(seed)
tf.compat.v1.set_random_seed(seed)
print(tf.__version__)

import pandas as pd
import seaborn as sns
from datetime import datetime
import matplotlib.pyplot as plt
plt.rc('font', size=16)
from sklearn.preprocessing import MinMaxScaler
import pandas as pd
import tensorflow as tf
from tensorflow.keras.layers import Input, Bidirectional, LSTM, Conv1D, Cropping1D
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import Callback

import numpy as np
import tensorflow as tf
from tensorflow.keras import layers as tfkl
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, mean_absolute_error


2.15.0


In [ ]:
# Define the ConvLSTM model function
def build_CONV_LSTM_model(input_shape, output_shape):
    input_layer = tfkl.Input(shape=input_shape, name='input_layer')
    x = tfkl.Bidirectional(tfkl.LSTM(64, return_sequences=True, name='lstm'), name='bidirectional_lstm')(input_layer)
    x = tfkl.Conv1D(128, 3, padding='same', activation='relu', name='conv')(x)
    output_layer = tfkl.Conv1D(output_shape[1], 3, padding='same', name='output_layer')(x)
    crop_size = output_layer.shape[1] - output_shape[0]
    output_layer = tfkl.Cropping1D((0, crop_size), name='cropping')(output_layer)
    model = tf.keras.Model(inputs=input_layer, outputs=output_layer, name='CONV_LSTM_model')
    model.compile(loss=tf.keras.losses.MeanSquaredError(), optimizer=tf.keras.optimizers.Adam())
    return model

In [ ]:
# Load your dataset
training_data = np.load('/content/gdrive/MyDrive/AN2DL_HW2/dataset/training_data.npy')
valid_periods = np.load('/content/gdrive/MyDrive/AN2DL_HW2/dataset/valid_periods.npy')
categories = np.load('/content/gdrive/MyDrive/AN2DL_HW2/dataset/categories.npy')


In [ ]:
# Preprocess the data: trim padding and categorize
trimmed_data = [series[start:end] for series, (start, end) in zip(training_data, valid_periods)]
data_by_category = {}
for series, category in zip(trimmed_data, categories):
    if category not in data_by_category:
        data_by_category[category] = []
    data_by_category[category].append(series)

In [ ]:
# Function to split each time series into training and test sets
test_set_proportion = 0.1  # e.g., last 10% of each series for testing
def split_train_test(series, test_set_proportion):
    test_set_size = int(len(series) * test_set_proportion)
    train_series = series[:-test_set_size] if test_set_size else series
    test_series = series[-test_set_size:] if test_set_size else []
    return train_series, test_series

In [ ]:
# Split data into training/validation and testing
train_data_by_category = {}
test_data_by_category = {}
for category, series_list in data_by_category.items():
    train_series_list = []
    test_series_list = []
    for series in series_list:
        train_series, test_series = split_train_test(series, test_set_proportion)
        train_series_list.append(train_series)
        test_series_list.append(test_series)
    train_data_by_category[category] = train_series_list
    test_data_by_category[category] = test_series_list

In [ ]:
# Define the sequence building function
def build_sequences(data, window, stride, telescope):
    X, y = [], []
    for series in data:
        if len(series) < window + telescope:
            continue  # Skip this series as it's too short
        for i in range(0, len(series) - window - telescope + 1, stride):
            X.append(series[i:i+window])
            y.append(series[i+window:i+window+telescope])
    X = np.array(X).reshape(-1, window, 1)  # Reshape for LSTM
    y = np.array(y).reshape(-1, telescope, 1)
    print(f"Shape of X: {X.shape}, Shape of y: {y.shape}")
    return X, y

In [ ]:
# Model training parameters
window_size = 30
stride = 10
forecast_horizon = 10

# Train a model for each category
models_by_category = {}
for category, series_list in train_data_by_category.items():
    X, y = build_sequences(series_list, window_size, stride, forecast_horizon)
    X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.1, random_state=42)
    input_shape = X_train.shape[1:]
    output_shape = y_train.shape[1:]
    model = build_CONV_LSTM_model(input_shape, output_shape)
    model.summary()
    model.fit(X_train, y_train, validation_data=(X_val, y_val), epochs=2, batch_size=64)
    models_by_category[category] = model
    print(f"Training model for category {category}")
    print(f"X_train shape: {X_train.shape}, y_train shape: {y_train.shape}")
    print(f"X_val shape: {X_val.shape}, y_val shape: {y_val.shape}")


# Generating predictions and calculating MSE and MAE for each category
# Check if Category F exists in test_data_by_category
print("Categories in test data:", test_data_by_category.keys())

Shape of X: (161869, 30, 1), Shape of y: (161869, 10, 1)
Model: "CONV_LSTM_model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_layer (InputLayer)    [(None, 30, 1)]           0         
                                                                 
 bidirectional_lstm (Bidire  (None, 30, 128)           33792     
 ctional)                                                        
                                                                 
 conv (Conv1D)               (None, 30, 128)           49280     
                                                                 
 output_layer (Conv1D)       (None, 30, 1)             385       
                                                                 
 cropping (Cropping1D)       (None, 10, 1)             0         
                                                                 
Total params: 83457 (326.00 KB)
Trainable params: 83457 (326

In [ ]:
# Check the length of data for Category F
if 'F' in test_data_by_category:
    print("Number of series in Category F:", len(test_data_by_category['F']))
    print("Lengths of series in Category F:", [len(series) for series in test_data_by_category['F']])

    # Generate sequences for Category F
    X_test_F, y_test_F = build_sequences(test_data_by_category['F'], window_size, stride, forecast_horizon)
    print("Generated sequences for Category F - X_test shape:", X_test_F.shape, "y_test shape:", y_test_F.shape)

    # If sequences are generated, check predictions
    if len(X_test_F) > 0 and len(y_test_F) > 0:
        predictions_F = models_by_category['F'].predict(X_test_F)
        print("Predictions for Category F shape:", predictions_F.shape)
    else:
        print("No sequences generated for Category F")


Number of series in Category F: 277
Lengths of series in Category F: [66, 77, 76, 65, 28, 15, 15, 15, 15, 15, 15, 15, 15, 15, 15, 16, 16, 16, 16, 12, 10, 21, 21, 21, 15, 21, 21, 21, 12, 21, 15, 21, 16, 10, 10, 21, 10, 22, 21, 21, 15, 15, 15, 15, 21, 16, 21, 21, 12, 21, 21, 21, 21, 65, 56, 52, 14, 33, 33, 31, 16, 16, 16, 40, 16, 16, 31, 27, 31, 5, 5, 4, 4, 6, 5, 5, 5, 4, 5, 5, 5, 4, 4, 3, 3, 3, 2, 3, 2, 4, 5, 5, 5, 5, 6, 5, 4, 4, 5, 4, 4, 5, 4, 6, 4, 3, 5, 5, 5, 42, 5, 8, 6, 8, 42, 6, 4, 11, 3, 7, 7, 4, 7, 4, 5, 3, 3, 2, 6, 2, 23, 4, 2, 2, 2, 3, 3, 2, 4, 17, 17, 17, 17, 17, 17, 21, 56, 21, 21, 21, 56, 21, 21, 21, 21, 21, 21, 21, 33, 32, 17, 17, 17, 17, 17, 21, 21, 21, 21, 52, 48, 48, 48, 48, 48, 34, 48, 34, 34, 34, 4, 22, 22, 22, 22, 20, 22, 21, 22, 22, 33, 106, 22, 22, 22, 22, 22, 48, 11, 11, 11, 11, 11, 11, 11, 26, 11, 11, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 23, 20, 20, 20, 20, 20, 20, 20, 20, 31, 31, 33, 33, 48, 33, 33, 52, 48, 48, 48, 48, 48, 17, 17, 17, 17, 17, 17, 17, 

In [ ]:
# Generating predictions and calculating MSE and MAE for each category
for category in test_data_by_category:
    X_test, y_test = build_sequences(test_data_by_category[category], window_size, stride, forecast_horizon)
    if len(X_test) > 0 and len(y_test) > 0:  # Ensure there is data for testing
        predictions = models_by_category[category].predict(X_test)

        # Flatten the arrays for comparison
        y_test_flattened = y_test.reshape(-1, y_test.shape[-1])
        predictions_flattened = predictions.reshape(-1, predictions.shape[-1])

        # Check shapes
        print(f"Category {category} - y_test shape: {y_test_flattened.shape}, predictions shape: {predictions_flattened.shape}")

        # Ensure the shapes are compatible
        if y_test_flattened.shape == predictions_flattened.shape:
            mse = mean_squared_error(y_test_flattened, predictions_flattened)
            mae = mean_absolute_error(y_test_flattened, predictions_flattened)
            print(f"Category {category} - MSE: {mse}, MAE: {mae}")
        else:
            print(f"Category {category} - Incompatible shapes for MSE and MAE calculation")


Shape of X: (1595, 30, 1), Shape of y: (1595, 10, 1)
50/50 [==============================] - 1s 6ms/step
Category D - y_test shape: (15950, 1), predictions shape: (15950, 1)
Category D - MSE: 0.006428571559690727, MAE: 0.056335587758133965
Shape of X: (535, 30, 1), Shape of y: (535, 10, 1)
17/17 [==============================] - 1s 8ms/step
Category E - y_test shape: (5350, 1), predictions shape: (5350, 1)
Category E - MSE: 0.00857773435740848, MAE: 0.05696567662274248
Shape of X: (1287, 30, 1), Shape of y: (1287, 10, 1)
41/41 [==============================] - 1s 5ms/step
Category A - y_test shape: (12870, 1), predictions shape: (12870, 1)
Category A - MSE: 0.007435225013449549, MAE: 0.0655347125189702
Shape of X: (1773, 30, 1), Shape of y: (1773, 10, 1)
56/56 [==============================] - 1s 5ms/step
Category C - y_test shape: (17730, 1), predictions shape: (17730, 1)
Category C - MSE: 0.012415862999155055, MAE: 0.0878175335239768
Shape of X: (793, 30, 1), Shape of y: (793, 10